In [14]:
import pandas as pd 
import numpy as np 
from openai import OpenAI
import json
import ast
import re
#from tqdm import tqdm 
#import matplotlib.pyplot as plt 
#import plotly.express as px

In [15]:
client = OpenAI(api_key=open("/Users/abdullahalibrahim/Desktop/openai_api.txt", "r").read().strip())

In [29]:
def custom_prompt(input_string):
    system_prompt = """Many experimental studies of team performance focus on specific tasks, raising questions about how findings generalize to other tasks.
    The lack of commensurability among tasks impedes generalization, as existing taxonomies don't comprehensively, consistently, and quantitatively measure task similarity. 
    """
    system_prompt += input_string
    system_prompt += """

    For each dimension, provide the name and a quantitative measure to operationalize it.
    YOUR ANSWER MUST FOLLOW THIS JSON FORMAT : {"dimensions": [{[DIMENSION 1 NAME] : [DIMENSION 1 OPERATIONALIZATION]}, {[DIMENSION 2 NAME] : [DIMENSION 2 OPERATIONALIZATION], ...]}
    """
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": system_prompt}
            ],
        temperature=0)
    return response

In [30]:
prompts_df = pd.read_csv('/Users/abdullahalibrahim/Repositories/llms_for_socsci/code/Direct Ask/Prompts.csv')
prompts_df

,Prompt Variation,Prompt,Characteristics of the Prompt
0,Detailed and Nuanced Dimensions,"To address this, create a nuanced set of dimen...",- Emphasis on detailed and nuanced dimensions-...
1,Broad and Generalizable Dimensions,"To address this, develop a broad set of dimens...",- Focus on broad and generalizable dimensions-...
2,Innovative and Creative Dimensions,"To address this, innovate a set of original di...",- Emphasis on innovative and creative dimensio...
3,Simple and Accessible Dimensions,"To address this, construct a simple set of dim...",- Focus on simplicity and accessibility - Stra...
4,Dynamic and Flexible Dimensions,"To address this, formulate a dynamic set of di...",- Emphasis on dynamic and flexible dimensions ...
5,Quantitative and Qualitative Blended Dimensions,"To address this, blend quantitative and qualit...",- Combination of quantitative and qualitative ...
6,Context-Specific and Tailored Dimensions,"To address this, design context-specific dimen...",- Emphasis on context-specific and tailored di...


In [31]:
def generate_responses(df):
    prompts_responses = []

    for prompt in df['Prompt']:
        response = custom_prompt(prompt)
        prompts_responses.append({
            "prompt": prompt,
            "response": response.choices[0].message.content
        })

    return prompts_responses

In [32]:
responses = generate_responses(prompts_df)
responses

[{'prompt': 'To address this, create a nuanced set of dimensions that capture subtle differences in tasks. Include detailed quantitative measures for each dimension, aiming for precise differentiation between tasks.',
  'response': '```json\n{\n  "dimensions": [\n    {\n      "Complexity": "Number of distinct steps or components required to complete the task."\n    },\n    {\n      "Interdependence": "Scale from 1 (completely independent) to 5 (highly interdependent), measuring the extent to which team members\' tasks are interconnected."\n    },\n    {\n      "Diversity of Skills Required": "Count of different specialized skills needed to perform the task effectively."\n    },\n    {\n      "Uncertainty": "Scale from 1 (low uncertainty) to 5 (high uncertainty), indicating the predictability of the task outcome based on the information available at the start."\n    },\n    {\n      "Time Pressure": "Measured in seconds/minutes/hours to deadline, indicating the urgency with which the ta

In [61]:

ast.literal_eval(responses[5]["response"].replace('\n', '')[7:-3])

{'dimensions': [{'Complexity': 'Number of distinct steps or components required to complete the task'},
  {'Interdependence': "Scale from 1 (low) to 5 (high) indicating the degree to which team members' tasks are interconnected"},
  {'Diversity of Skills Required': 'Count of different specialized skills needed to perform the task effectively'},
  {'Uncertainty': 'Scale from 1 (low) to 5 (high) representing the predictability of task outcomes'},
  {'Time Pressure': 'Time given to complete the task measured in minutes or hours'},
  {'Task Novelty': 'Scale from 1 (familiar) to 5 (novel) indicating how common or new the task is to the team members'},
  {'Information Load': 'Amount of information (in bytes or number of items) that must be processed to complete the task'},
  {'Goal Clarity': 'Scale from 1 (ambiguous) to 5 (clear) indicating how well-defined the task goals are'},
  {'Physical Demand': 'Scale from 1 (low) to 5 (high) indicating the level of physical effort required'},
  {'Coor

In [51]:
parsed_data = []
for response in responses:
    # Extract the prompt for the current response
    prompt = response["prompt"]
    # The replace function is used to clean up the string, removing backticks and excess whitespace
    cleaned_response = response["response"].replace('\n', '')[7:-3]
    # Use ast.literal_eval to safely evaluate the string into a Python object
    dimension_data = ast.literal_eval(cleaned_response)

    # Ensure dimension_data is a list of dictionaries before iterating
    if isinstance(dimension_data, dict) and "dimensions" in dimension_data:
        for dimension_dict in dimension_data["dimensions"]:
            if isinstance(dimension_dict, dict):
                for dimension_name, dimension_description in dimension_dict.items():
                    parsed_data.append({
                        "Prompt": prompt,
                        "Dimension Name": dimension_name,
                        "Dimension Description": dimension_description
                    })
# Create a DataFrame from the parsed data
dimensions_df = pd.DataFrame(parsed_data)

# Display the first few rows of the DataFrame to verify
dimensions_df.head()


,Prompt,Dimension Name,Dimension Description
0,"To address this, create a nuanced set of dimen...",Complexity,Number of distinct steps or components require...
1,"To address this, create a nuanced set of dimen...",Interdependence,Scale from 1 (completely independent) to 5 (hi...
2,"To address this, create a nuanced set of dimen...",Diversity of Skills Required,Count of different specialized skills needed t...
3,"To address this, create a nuanced set of dimen...",Uncertainty,Scale from 1 (low uncertainty) to 5 (high unce...
4,"To address this, create a nuanced set of dimen...",Time Pressure,"Measured in seconds/minutes/hours to deadline,..."


In [45]:
#response["prompt"]
response["response"].replace('```json', '').replace('```', '').replace('\n', '').strip()

'{  "dimensions": [    {      "Complexity": "Number of distinct steps or components required to complete the task."    },    {      "Interdependence": "Scale from 1 (completely independent) to 5 (highly interdependent), measuring the extent to which team members\' tasks are interconnected."    },    {      "Diversity of Skills Required": "Count of different specialized skills needed to perform the task effectively."    },    {      "Uncertainty": "Scale from 1 (low uncertainty) to 5 (high uncertainty), indicating the predictability of the task outcome based on the information available at the start."    },    {      "Time Pressure": "Measured in seconds/minutes/hours to deadline, indicating the urgency with which the task must be completed."    },    {      "Information Load": "Quantified as the number of informational units that must be processed to complete the task."    },    {      "Problem-Solving Demand": "Scale from 1 (routine task) to 5 (high problem-solving demand), reflecting

In [56]:
#Print out each dimensions by prompt
dfs_per_prompt = {} # Initialize a dictionary to hold the DataFrames for each unique prompt

# Iterate over the grouped data to create a DataFrame for each unique prompt
for prompt, dimensions in grouped_dimensions.items():
    df = pd.DataFrame(dimensions) # Convert the list of dimension dictionaries to a DataFrame
    dfs_per_prompt[prompt] = df # Add the DataFrame to the dictionary with the prompt as the key

i = 1
for prompt, df in dfs_per_prompt.items(): # Display the DataFrame for each prompt to verify
    print(f"Prompt {i}: {prompt}")
    display(df) 
    i= i+1


Prompt 1: To address this, blend quantitative and qualitative aspects to create a comprehensive set of dimensions. Include a mix of numerical measures and descriptive indicators for a holistic task comparison.


,Dimension Name,Dimension Description
0,Complexity,Number of distinct steps or components require...
1,Interdependence,Scale from 1 (low) to 5 (high) indicating the ...
2,Diversity of Skills Required,Count of different specialized skills needed t...
3,Uncertainty,Scale from 1 (low) to 5 (high) representing th...
4,Time Pressure,Time given to complete the task measured in mi...
5,Task Novelty,Scale from 1 (familiar) to 5 (novel) indicatin...
6,Information Load,Amount of information (in bytes or number of i...
7,Goal Clarity,Scale from 1 (ambiguous) to 5 (clear) indicati...
8,Physical Demand,Scale from 1 (low) to 5 (high) indicating the ...
9,Coordination Requirement,Number of coordination points where team membe...


Prompt 2: To address this, construct a simple set of dimensions that are easily understandable and accessible. Include straightforward quantitative measures to operationalize these dimensions, suitable for a wide audience.


,Dimension Name,Dimension Description
0,Complexity,Number of distinct steps or actions required t...
1,Interdependence,Scale from 0 (completely independent) to 1 (co...
2,Diversity of Skills,Number of different specialized skills or know...
3,Time Pressure,Time available to complete the task divided by...
4,Uncertainty,"Scale from 0 (no uncertainty, outcome is predi..."
5,Goal Clarity,Scale from 0 (goals are ambiguous and not clea...
6,Information Availability,Ratio of the amount of information available t...
7,Task Novelty,Scale from 0 (task is routine and familiar to ...


Prompt 3: To address this, create a nuanced set of dimensions that capture subtle differences in tasks. Include detailed quantitative measures for each dimension, aiming for precise differentiation between tasks.


...

,Dimension Name,Dimension Description
0,Complexity,Number of distinct steps or components require...
1,Interdependence,Scale from 0 (completely independent tasks) to...
2,Diversity of Skills Required,Variety index based on the number of different...
3,Uncertainty,"Scale from 0 (no uncertainty, outcome is predi..."
4,Time Pressure,Measured as the inverse of the amount of time ...
5,Goal Clarity,Scale from 0 (goals are ambiguous and not clea...
6,Information Load,Quantified as the amount of information units ...
7,Task Novelty,Scale from 0 (task is routine and familiar) to...
8,Physical Demand,Measured by the average metabolic equivalent (...
9,Coordination Demand,Scale from 0 (no coordination required) to 1 (...


Prompt 6: To address this, formulate a dynamic set of dimensions that can adapt to changing task environments. Provide flexible quantitative measures that allow for adjustment and modification in different contexts.


,Dimension Name,Dimension Description
0,Complexity,{'description': 'The number of distinct elemen...
1,Uncertainty,{'description': 'The unpredictability associat...
2,Duration,{'description': 'The length of time required t...
3,Novelty,{'description': 'The degree to which the task ...
4,Coordination Demand,{'description': 'The extent to which the task ...
5,Cognitive Load,{'description': 'The mental effort required to...
6,Physical Demand,{'description': 'The physical effort required ...
7,Skill Variety,{'description': 'The range of skills and abili...
8,Goal Clarity,{'description': 'The degree to which the goals...
9,Feedback Loop,{'description': 'The mechanism by which inform...


Prompt 7: To address this, innovate a set of original dimensions that offer a fresh perspective on task classification. Ensure the dimensions and their quantitative measures are novel and insightful for comparing tasks.


,Dimension Name,Dimension Description
0,Cognitive Complexity,{'Operationalization': 'A numerical score base...
1,Interdependence Level,{'Operationalization': 'A scale from 0 to 1 wh...
2,Temporal Dynamics,{'Operationalization': 'A measure of the task'...
3,Innovation Requirement,{'Operationalization': 'A score derived from t...
4,Physical Coordination,{'Operationalization': 'A metric based on the ...
5,Environmental Variability,{'Operationalization': 'A variability index ca...
6,Resource Constraints,{'Operationalization': 'A constraint score bas...
7,Stress Level,{'Operationalization': 'A stress index measure...
8,Learning Curve,{'Operationalization': 'A learning index calcu...
9,Outcome Measurability,{'Operationalization': 'A measurability score ...
